In [ ]:
优先队列
思路和算法

我们用两个优先队列 queMax 和 queMin 分别记录大于中位数的数和小于等于中位数的数。当累计添加的数的数量为奇数时，queMin 中的数的数量比 queMax 多一个，此时中位数为 queMin 的队头。当累计添加的数的数量为偶数时，两个优先队列中的数的数量相同，此时中位数为它们的队头的平均值。

当我们尝试添加一个数 num 到数据结构中，我们需要分情况讨论：

num≤max{queMin}

此时 num 小于等于中位数，我们需要将该数添加到 queMin 中。新的中位数将小于等于原来的中位数，因此我们可能需要将 queMin 中最大的数移动到 queMax 中。

num>max{queMin}

此时 num 大于中位数，我们需要将该数添加到 queMin 中。新的中位数将大于等于原来的中位数，因此我们可能需要将 queMax 中最小的数移动到 queMin 中。

特别地，当累计添加的数的数量为 0 时，我们将 num 添加到 queMin 中。

时间复杂度：

addNum: O(logn)，其中 n 为累计添加的数的数量。
findMedian: O(1)。
空间复杂度：O(n)，主要为优先队列的开销。



In [ ]:
class MedianFinder:

    def __init__(self):
        self.queMin = list()
        self.queMax = list()

    def addNum(self, num: int) -> None:
        queMin_ = self.queMin
        queMax_ = self.queMax

        if not queMin_ or num <= -queMin_[0]:
            heapq.heappush(queMin_, -num)
            if len(queMax_) + 1 < len(queMin_):
                heapq.heappush(queMax_, -heapq.heappop(queMin_))
        else:
            heapq.heappush(queMax_, num)
            if len(queMax_) > len(queMin_):
                heapq.heappush(queMin_, -heapq.heappop(queMax_))
        
    def findMedian(self) -> float:
        queMin_ = self.queMin
        queMax_ = self.queMax

        if len(queMin_) > len(queMax_):
            return -queMin_[0]
        return (-queMin_[0] + queMax_[0]) / 2


In [ ]:
有序集合 + 双指针
我们也可以使用有序集合维护这些数。我们把有序集合看作自动排序的数组，使用双指针指向有序集合中的中位数元素即可。当累计添加的数的数量为奇数时，双指针指向同一个元素。当累计添加的数的数量为偶数时，双指针分别指向构成中位数的两个数。

当我们尝试添加一个数 \textit{num}num 到数据结构中，我们需要分情况讨论：

初始有序集合为空时，我们直接让左右指针指向 \textit{num}num 所在的位置。

有序集合为中元素为奇数时，\textit{left}left 和 \textit{right}right 同时指向中位数。如果 \textit{num}num 大于等于中位数，那么只要让 \textit{left}left 左移，否则让 \textit{right}right 右移即可。

有序集合为中元素为偶数时，\textit{left}left 和 \textit{right}right 分别指向构成中位数的两个数。

当 \textit{num}num 成为新的唯一的中位数，那么我们让 \textit{left}left 右移，\textit{right}right 左移，这样它们即可指向 \textit{num}num 所在的位置；
当 \textit{num}num 大于等于 \textit{right}right，那么我们让 \textit{left}left 右移即可；
当 \textit{num}num 小于 \textit{right}right 指向的值，那么我们让 \textit{right}right 左移，注意到如果 \textit{num}num 恰等于 \textit{left}left 指向的值，那么 \textit{num}num 将被插入到 \textit{left}left 右侧，使得 \textit{left}left 和 \textit{right}right 间距增大，所以我们还需要额外让 \textit{left}left 指向移动后的 \textit{right}right。

时间复杂度：

\textit{addNum}addNum: O(\log n)O(logn)，其中 nn 为累计添加的数的数量。
\textit{findMedian}findMedian: O(1)O(1)。
空间复杂度：O(n)O(n)，主要为有序集合的开销。

进阶 1
如果数据流中所有整数都在 00 到 100100 范围内，那么我们可以利用计数排序统计每一类数的数量，并使用双指针维护中位数。

进阶 2
如果数据流中 99\%99% 的整数都在 00 到 100100 范围内，那么我们依然利用计数排序统计每一类数的数量，并使用双指针维护中位数。对于超出范围的数，我们可以单独进行处理，建立两个数组，分别记录小于 00 的部分的数的数量和大于 100100 的部分的数的数量即可。当小部分时间，中位数不落在区间 [0,100][0,100] 中时，我们在对应的数组中暴力检查即可。


In [ ]:
from sortedcontainers import SortedList

class MedianFinder:

    def __init__(self):
        self.nums = SortedList()
        self.left = self.right = None
        self.left_value = self.right_value = None

    def addNum(self, num: int) -> None:
        nums_ = self.nums

        n = len(nums_)
        nums_.add(num)

        if n == 0:
            self.left = self.right = 0
        else:
            # 模拟双指针，当 num 小于 self.left 或 self.right 指向的元素时，num 的加入会导致对应指针向右移动一个位置
            if num < self.left_value:
                self.left += 1
            if num < self.right_value:
                self.right += 1

            if n & 1:
                if num < self.left_value:
                    self.left -= 1
                else:
                    self.right += 1
            else:
                if self.left_value < num < self.right_value:
                    self.left += 1
                    self.right -= 1
                elif num >= self.right_value:
                    self.left += 1
                else:
                    self.right -= 1
                    self.left = self.right
        
        self.left_value = nums_[self.left]
        self.right_value = nums_[self.right]

    def findMedian(self) -> float:
        return (self.left_value + self.right_value) / 2


In [15]:
# 2236 ms	34.2 MB
class MedianFinder:

    def __init__(self):
        """
        initialize your data structure here.
        """
        self.listt = []


    def addNum(self, num: int) -> None:
        self.listt.append(num)

    def findMedian(self) -> float:
        n = len(self.listt)
        if n % 2 == 0:
            return (self.listt[n//2]+self.listt[n//2-1])/2
        else:
            return self.listt[n//2]

In [19]:
a = MedianFinder()

In [20]:
a.addNum(6)
a.addNum(10)
a.addNum(2)
a.addNum(6)

In [21]:
a.findMedian()

6.0

In [22]:
a = [1,2,6,7,5,4,]

In [23]:
a.sort()

In [24]:
a

[1, 2, 4, 5, 6, 7]

In [ ]:
["MedianFinder","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian"]
[[],[6],[],[10],[],[2],[],[6],[],[5],[],[0],[],[6],[],[3],[],[1],[],[0],[],[0],[]]

In [ ]:
[null,null,6.0,null,8.0,null,6.0,null,6.0,null,6.0,null,5.5,null,6.0,null,5.5,null,5.0,null,4.0,null,3.0]

In [9]:
[null,null,6.00000,null,8.00000,null,10.00000,null,6.00000,null,2.00000,null,4.00000,null,6.00000,null,5.50000,null,5.00000,null,2.50000,null,0.00000]

1.5